In [1]:
import os 
import requests 
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown , display
from openai import OpenAI

In [4]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

# Checking the key
if not api_key:
    print("No API key was found")
elif not api_key.startswith("sk-proj-"):
    print("API key was found but it does not look correct")
elif api_key.strip() != api_key:
    print("API key was found butt there are blank spaces in between")
else:
    print("API key is found and looks good to go")

API key is found and looks good to go


In [5]:
openai = OpenAI()

In [6]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [11]:
apple = Website("https://en.wikipedia.org/wiki/Apple_Inc.")
print(apple.title)
print(apple.text)

Apple Inc. - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Special pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
History
Toggle History subsection
1.1
1976–1980: Founding and incorporation
1.2
1980–1990: Success with Macintosh
1.3
1990–1997: Decline and restructuring
1.4
1997–2007: Return to profitability
1.5
2007–2011: Success with mobile devices
1.6
2011–present: Post-Jobs era, Tim Cook
2
Products
Toggle Products subsection
2.1
Mac
2.2
iPhone
2.3
iPad
2.4
Other products
2.5
Services
3
Marketing
Toggle Marketing subsection
3.1
Branding
3.2
Advertising
3.3
Stores
3.4
Market power
3.5
Privacy
4
Corporate affairs
Toggle Corporate affairs subsection
4.1
Busi

In [12]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [13]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [14]:
print(user_prompt_for(apple))

You are looking at a website titled Apple Inc. - Wikipedia
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Special pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
History
Toggle History subsection
1.1
1976–1980: Founding and incorporation
1.2
1980–1990: Success with Macintosh
1.3
1990–1997: Decline and restructuring
1.4
1997–2007: Return to profitability
1.5
2007–2011: Success with mobile devices
1.6
2011–present: Post-Jobs era, Tim Cook
2
Products
Toggle Products subsection
2.1
Mac
2.2
iPhone
2.3
iPad


In [15]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [16]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [17]:
summarize("https://en.wikipedia.org/wiki/Apple_Inc.")

'# Summary of Apple Inc. - Wikipedia\n\nApple Inc. is an American multinational technology company headquartered in Cupertino, California. Founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne, Apple started as a personal computer manufacturer and has since evolved into a major player in consumer electronics, software, and services.\n\n## Key Historical Milestones\n- **Founding and Early Years (1976-1980)**: Apple began with the introduction of the Apple I personal computer, followed by the successful Apple II.\n- **1980s Success and Challenges**: The launch of the Macintosh in 1984 marked a pivotal moment, introducing the first mass-market computer with a graphical user interface.\n- **Struggles and Restructuring (1990-1997)**: After a downturn and loss of market share, Apple faced near bankruptcy, which led to the return of Steve Jobs after acquiring NeXT.\n- **Resurgence and Innovation (1997-2011)**: Under Jobs’ leadership, Apple introduced highly successful produ

In [18]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [19]:
display_summary("https://en.wikipedia.org/wiki/Apple_Inc.")

# Summary of Apple Inc. - Wikipedia

Apple Inc. is a prominent American multinational technology company headquartered in Cupertino, California. Founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne, Apple started with the Apple I personal computer and gained fame with the Apple II, Macintosh, and later products such as the iPod, iPhone, and iPad. The company has evolved from a computer manufacturer to a major player in consumer electronics, software, and services.

## Key Highlights

- **History**: Apple has gone through various phases since its inception, including significant successes and downturns. It experienced a revival in the late 1990s with the return of Steve Jobs, leading to products like the iMac and iPod.
- **Financial Performance**: As of fiscal year 2024, Apple reported revenues of approximately $391 billion, showcasing strong growth in its smartphone (iPhone) and services sectors.
- **Products and Services**: The company's product line includes:
  - **Mac Computers**: iMac, MacBook, Mac Pro
  - **iPhone**: A major revenue source and the first smartphone to revolutionize the market.
  - **iPad**: Leading in tablet sales globally.
  - **Wearables**: Includes the Apple Watch and AirPods.
  - **Services**: Notable offerings include the App Store, Apple Music, Apple TV+, and iCloud.
  
## Recent Developments
- In March 2023, Apple launched a "buy now, pay later" service called 'Apple Pay Later'.
- In November 2023, Apple reached a settlement of $25 million over allegations of discrimination in hiring practices.
- In January 2024, the company announced compliance with new EU competition laws, which allowed alternative app stores on its platform.

## Environmental and Social Responsibility
Apple has made commitments toward sustainability, including a goal to become carbon neutral by 2030 across its supply chain. However, it has faced criticism regarding labor practices within its supply chain and environmental impacts related to sourcing materials.

Overall, Apple remains one of the most valuable brands globally, known for its innovative products and strong consumer loyalty, with significant annual revenues primarily driven by iPhone sales and a rapidly expanding services sector.